In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    api_key=api_key,
    model="mixtral-8x7b-32768",
    temperature=0.5
)

In [5]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000242D2DCF9D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000242D2DDE700>, temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [11]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [12]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker.Given the above text, it is your job to create a quiz of {number} multiple choice question for {subject} student in {tone} tone. Make Sure the questions are not repeated and check all the questions for {subject} student in {tone} tone. Make sure the question are not repeated and check all the questions to be confoming the text as well. Make sure to format your response like response_json below and use it as a guide. Ensure to make {number} MCQs.

## response_json
{response_json}
"""

In [13]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone"],
    template=TEMPLATE,
)

In [14]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_template = PromptTemplate(
    input_variables=["quiz", "subject"],
    template=TEMPLATE2
)

In [18]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_template, output_key="review", verbose=True)

In [19]:

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [20]:
file_path = "../data.txt"

In [21]:
with open(file_path, "r") as f:
    TEXT = f.read()

In [22]:
TEXT

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [23]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [24]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [25]:

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Suraj\AppData\Local\Temp\ipykernel_35640\3602604365.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for biology students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{
"1": {
"mcq": "What is the overarching theme that connects all of biology as a single field?",
"options": {
"a": "The study of cells and genes",
"b": "Evolution and energy processing",
"c": "Anatomy and physiology of plants and animals",
"d": "Classification of prokaryotic and eukaryotic organisms"
},
"correct": "b"
},
"2": {
"mcq": "Which of the following is NOT a level of organization that biologists study?",
"options": {
"a": "Molecular biology of a cell",
"b": "Anatomy and physiolo

In [26]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [28]:
quiz=response.get("quiz")

In [30]:
quiz=json.loads(quiz)

In [31]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [32]:
quiz_table_data

[{'MCQ': 'What is the overarching theme that connects all of biology as a single field?',
  'Choices': 'a: The study of cells and genes | b: Evolution and energy processing | c: Anatomy and physiology of plants and animals | d: Classification of prokaryotic and eukaryotic organisms',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a level of organization that biologists study?',
  'Choices': 'a: Molecular biology of a cell | b: Anatomy and physiology of plants and animals | c: Energy processing in an ecosystem | d: Cellular structure and function',
  'Correct': 'c'},
 {'MCQ': 'What is the name of the scientific method used by biologists to study the world around them?',
  'Choices': 'a: Hypothesis testing | b: Observation and experimentation | c: Data analysis and interpretation | d: Model building and simulation',
  'Correct': 'b'},
 {'MCQ': 'What is the name of the process that explains the unity and diversity of life?',
  'Choices': 'a: Cellular respiration | b: Gene tran

In [35]:
quiz = pd.DataFrame(quiz_table_data)

In [36]:
quiz.to_csv("machine_learning_quiz.csv", index=False)